In [1]:
import tensorflow as tf
slim = tf.contrib.slim
from PIL import Image
from vgg import *
import numpy as np
import requests
from io import BytesIO
#from math import 
import time

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

%matplotlib inline

In [3]:
import pickle as pkl

In [4]:
id_img_dict = pkl.load(open('id_img3.p', 'rb'))

In [23]:
get_image(0, ('https://img00.deviantart.net/9240/i/2017/073/a/1/rhys_alder_by_nanocolors-db2a39s.png', 0))

> <ipython-input-22-a9e6c37eceaf>(20)get_image()
-> width, height = img.size
(Pdb) img
<PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1600x1011 at 0x11C371DA0>
(Pdb) img.shape
*** AttributeError: 'PngImageFile' object has no attribute 'shape'
(Pdb) img.mode
'RGBA'
(Pdb) img.mode == 'RGB'
False
(Pdb) img.mode == 'RGBA'
True
(Pdb)         tmp = Image.new("RGB", img.size, (255, 255, 255))
(Pdb)         tmp.paste(img, mask=img.split()[3]) # 3 is the alpha channel
(Pdb) tmp.show()
(Pdb) tmp
<PIL.Image.Image image mode=RGB size=1600x1011 at 0x11C3711D0>
(Pdb) quit


BdbQuit: 

In [25]:
def get_image(_id, data):

    url, artist = data
    
    response = requests.get(url)
    count = 0
    while response.status_code != 200:
        if count > 25: return _id, None, None
        time.sleep(1.0)
        response = requests.get(url)
        count += 1

    try:
        img = Image.open(BytesIO(response.content))
    except OSError:
        import pdb; pdb.set_trace()
        return _id, None, None

       
    if img.mode == 'RGBA':
        tmp = Image.new("RGB", img.size, (255, 255, 255))
        tmp.paste(img, mask=img.split()[3]) # 3 is the alpha channel
        img = tmp

    
    width, height = img.size
    half_width, half_height = int(width/2), int(height/2)
    offset = min(half_width, half_height)
    
    sq_im = img.crop((half_width-offset, half_height-offset, half_width+offset, half_height+offset))
    #q_im.thumbnail((224, 224))


    #convert image
    sq_im = sq_im.resize((224, 224), Image.BICUBIC)
        
        
    im = np.array(sq_im, dtype='float32')
    try: 
        im = im.reshape(-1,224,224,3)
    except ValueError:
        import pdb; pdb.set_trace()
        return _id, None, None
    
    return _id, im, artist


In [6]:
checkpoint_file = './vgg_16.ckpt'

In [7]:
input_tensor = tf.placeholder(tf.float32, shape=(None,224,224,3), name='input_image')
scaled_input_tensor = tf.scalar_mul((1.0/255), input_tensor)
scaled_input_tensor = tf.subtract(scaled_input_tensor, 0.5)
scaled_input_tensor = tf.multiply(scaled_input_tensor, 2.0)

In [8]:
sess = tf.Session()
arg_scope = vgg_arg_scope()
with slim.arg_scope(arg_scope):
    _, end_points = vgg_16(scaled_input_tensor, is_training=False)
saver = tf.train.Saver()
saver.restore(sess, checkpoint_file)

In [9]:
id_feature_dict = pkl.load(open('id_feature_dict_with_artist3.p', 'rb'))

In [11]:
id_img_list = list(id_img_dict.items())
id_img_list = [x for x in id_img_list if x[0] not in id_feature_dict]

In [13]:
from tqdm import tqdm_notebook as tqdm

In [ ]:
for _id, data in tqdm(id_img_list):
    if _id in id_feature_dict: continue
    _, im, artist = get_image(_id, data)
    if im is None:
        continue;
    feats = np.squeeze(sess.run(end_points['vgg_16/fc7'], feed_dict={input_tensor: im}))
    id_feature_dict[_id] = (feats, artist)


> <ipython-input-25-654294ff3949>(43)get_image()
-> return _id, None, None
(Pdb) img
<PIL.PngImagePlugin.PngImageFile image mode=L size=1280x894 at 0x11010D7B8>
(Pdb) im
array([[ 195.,  195.,  195., ...,  195.,  195.,  195.],
       [ 195.,  195.,  195., ...,  195.,  195.,  195.],
       [ 195.,  195.,  195., ...,  195.,  195.,  195.],
       ..., 
       [ 195.,  195.,  195., ...,  195.,  195.,  195.],
       [ 195.,  195.,  195., ...,  195.,  195.,  195.],
       [ 195.,  195.,  195., ...,  195.,  195.,  195.]], dtype=float32)
(Pdb) im.shape
(224, 224)
(Pdb) continue

Program interrupted. (Use 'cont' to resume).
--Call--
> /usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/contextlib.py(86)__exit__()
-> def __exit__(self, type, value, traceback):


In [23]:
pkl.dump(id_feature_dict, open('id_feature_dict_with_artist32.p', 'wb'))

In [ ]:
sess.close()